
### Expert Question Answerer for InsureLLM

LangChain 1.0 implementation of a RAG pipeline.

Using the VectorStore we created last time (with HuggingFace `all-MiniLM-L6-v2`)

In [8]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings
import gradio as gr

In [4]:

MODEL = "openai/gpt-oss-120b"
DB_NAME = "vector_db"
load_dotenv(override=True)

True

### Connect to Chroma; use Hugging Face all-MiniLM-L6-v2

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)

### Set up the 2 key LangChain objects: retriever and llm

#### A sidebar on "temperature":
- Controls how diverse the output is
- A temperature of 0 means that the output should be predictable
- Higher temperature for more variety in answers

- It actually controls which tokens get selected during inference
- temperature=0 means: always select the token with highest probability
- temperature=1 usually means: a token with 10% probability should be picked 10% of the time

Note: a temperature of 0 doesn't mean outputs will always be reproducible. You also need to set a random seed. We will do that in weeks 6-8. (Even then, it's not always reproducible.)

Note 2: if you want creativity, use the System Prompt!

In [9]:
retriever = vectorstore.as_retriever()
llm = ChatGroq(temperature=0, model_name=MODEL)

### These LangChain objects implement the method `invoke()`

In [10]:
retriever.invoke("Who is Avery?")

[Document(id='db28f34b-8fff-41c9-b40d-9e5484110d81', metadata={'doc_type': 'employees', 'source': 'knowledge-base/employees/Avery Lancaster.md'}, page_content="## Other HR Notes\n- **Professional Development**: Avery has actively participated in leadership training programs and industry conferences, representing Insurellm and fostering partnerships.  \n- **Diversity & Inclusion Initiatives**: Avery has championed a commitment to diversity in hiring practices, seeing visible improvements in team representation since 2021.  \n- **Work-Life Balance**: Feedback revealed concerns regarding work-life balance, which Avery has approached by implementing flexible working conditions and ensuring regular check-ins with the team.\n- **Community Engagement**: Avery led community outreach efforts, focusing on financial literacy programs, particularly aimed at underserved populations, improving Insurellm's corporate social responsibility image.  \n\nAvery Lancaster has demonstrated resilience and ada

In [11]:
llm.invoke("Who is Avery?")

AIMessage(content='I’m not sure which “Avery” you’re referring to—there are many notable people, fictional characters, and public figures with that name. Could you give me a bit more context (e.g., a last name, a field like music, sports, literature, a TV show, etc.) so I can provide the most relevant information?', additional_kwargs={'reasoning_content': 'The user asks "Who is Avery?" No context. Could be a public figure, a character, etc. Need to ask clarifying? Could give general answer: Avery could be a name, many people. Could ask for context. According to policy, we can ask for clarification. So respond asking for more context.'}, response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 75, 'total_tokens': 219, 'completion_time': 0.306751493, 'completion_tokens_details': {'reasoning_tokens': 65}, 'prompt_time': 0.003020099, 'prompt_tokens_details': None, 'queue_time': 0.051566311, 'total_time': 0.309771592}, 'model_name': 'openai/gpt-oss-120b', 'system_finger

## Time to put this together!

In [12]:
SYSTEM_PROMPT_TEMPLATE = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
If relevant, use the given context to answer any question.
If you don't know the answer, say so.
Context:
{context}
"""

In [13]:
def answer_question(question: str, history):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
    response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
    return response.content

In [14]:
answer_question("Who is Averi Lancaster?", [])

'**Avery\u202fLancaster** (you might have meant “Averi” – it’s a common typo) is one of the founding pillars of Insurellm.\n\n| Detail | Information |\n|--------|--------------|\n| **Full Name** | Avery\u202fLancaster |\n| **Date of Birth** | March\u202f15\u202f,\u202f1985 |\n| **Current Role** | Co‑Founder & Chief Executive Officer (CEO) |\n| **Location** | San\u202fFrancisco, California |\n| **Current Salary** | $225,000 per year |\n\n### Career Highlights at Insurellm\n- **2015 – Present:** Co‑Founded Insurellm and has served as CEO since day one. Under her leadership, the company has grown into a leading Insurance‑Tech provider, known for innovative products and a strong market presence.\n- **Prior Experience (2013‑2015):** Senior Product Manager at Innovate Insurance Solutions, where she created groundbreaking insurance products for the tech sector—experience that helped shape Insurellm’s early vision.\n\n### What Makes Her Stand Out\n- **Innovative Leadership:** Avery is celebrat

In [15]:
gr.ChatInterface(answer_question).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [17]:
pip install litellm 

Note: you may need to restart the kernel to use updated packages.
